

Used `pip` here as assumed connecting to a local jupyter server, if using own environment then have your own env command to have `kafka-python` installed. This works for the pip env as defined in `envs/pip-venvs/requirements-aiven-demo.txt`

Conda might not work, had some issue with the version of kafka-python and didn't want to get into debugging.

In [ ]:
%%bash
pip install kafka-python

In [ ]:
import json
from kafka import KafkaProducer

In [ ]:
# folderName = "~/kafkaCerts/kafka-pizza/"
producer = KafkaProducer(
    bootstrap_servers="redpanda-0:9092",
    # security_protocol="SSL",
    # ssl_cafile=folderName+"ca.pem",
    # ssl_certfile=folderName+"service.cert",
    # ssl_keyfile=folderName+"service.key",
    value_serializer=lambda v: json.dumps(v).encode('ascii'),
    key_serializer=lambda v: json.dumps(v).encode('ascii')

)

In [ ]:
producer.send("test-topic",
                key={"key": 1},
                value={"message": "hello world"}
            )
producer.flush()

# Create a data generator using Faker
[Faker](https://faker.readthedocs.io/en/master/) is a Python library.
Going to use that now to have better data generation.

In [ ]:
pip install Faker

In [ ]:
from faker import Faker

fake = Faker()

In [ ]:

# message is a tuple
message = {
    'name': fake.name(),
    'address': fake.address(),
    'phone': fake.phone_number()
}
print(message)

## Creating a custom data provider
Faker's [standard providers](https://faker.readthedocs.io/en/master/providers.html) do not include a pizza generator, but we can [create our own](https://faker.readthedocs.io/en/master/providers.html).

In [ ]:
import sys
sys.path.append("/home/jovyan/work/data-gen/") # add path for below import

from pizzaProducer import PizzaProvider # see pizzaProducer.py

fake.add_provider(PizzaProvider)

for i in range(0,10):
    print(fake.pizzaName())
    print(fake.pizzaTopping())
    print(fake.pizzaShop())
    print("\n")

## Creating an order
Name, address, phone number, 1-10 pizzas, 0-5 toppings, randomly generate orderId, return the message and key. Keyed on shop name (to ensure partition assignment).

In [ ]:
import random
MAX_PIZZAS_IN_ORDER = 10
MAX_ADDITIONAL_TOPPINGS_IN_PIZZA = 5



def producePizzaOrder(orderId = 1):
    shop = fake.pizzaShop()
    pizzas = []
    for pizza in range(random.randint(1,MAX_PIZZAS_IN_ORDER)):
        toppings = []
        for topping in range(random.randint(1,MAX_ADDITIONAL_TOPPINGS_IN_PIZZA)):
            toppings.append(fake.pizzaTopping())
        pizzas.append({
            "pizzaName": fake.pizzaName(),
            "additionalToppings": toppings
        })
    
    message = {
        "id": orderId,
        "shop": shop,
        "customerName": fake.unique.name(),
        "phoneNumber": fake.unique.phone_number(),
        "address": fake.unique.address(),
        "pizzas": pizzas
    }

    key = shop

    return message, key

In [ ]:
producePizzaOrder(704)

# Creating a fake producer

In [ ]:
import time
topic_name = "stream_test_two"
num_messages = 100
i = 0


while i < num_messages:
    message, key = producePizzaOrder(i)

    print("Sending: {}".format(message))

    producer.send(
        topic_name,
        key = key,
        value = message
                  )
    
    time.sleep(2)

    if (i % num_messages) == 0:
        producer.flush()
    i = i + 1

producer.flush()